<h1 style="color:navy">Изучение заведений общественного питания Москвы</h1> 

<div style ="color:navy; font-size:16pt"><b>Описание проекта</b></div>

- Заказчик планирует открыть заведение общественного питания в Москве: кафе, ресторан, пиццерию, паб или бар.  Тип, расположение, меню и цены пока не определены. 
- Наиболее привлекательной идеей для клиента кажется открытие дорогой кофейни.

Для анализа имеются данные о заведениях общественного питания Москвы составленные с помощью сервисов «Яндекс Карты» и «Яндекс Бизнес» на лето 2022 года.

Требуется исследовать этот рынок, найти интересные особенности и презентовать полученные результаты, которые помогут в выборе подходящего места. Особое внимание следует уделить изучению кофеен, чтобы понять осуществимо ли желание клиента.

**Цели проекта**

- научиться представлять имеющуюся информацию в удобном и понятном виде с помощью различных способов визуализации;
- приобрести навыки работы с геоданными;
- создать презентацию по результатам исследования данных.

**Ход исследования**

1. Подготовить данные к исследованию - исключить дубликаты, по возможности заполнить пропущенные значения.
2. Провести исследовательский анализ данных для сравнения заведений общественного питания по категориям и расположению.
3. Отдельно изучить кофейни. Найти подходящие ниши в этом сегменте рынка. 
4. Визуализировать результаты исследования.
5. Подготовить презентацию для клиента по результатам анализа. 

**Презентация** доступна по ссылке: https://drive.google.com/file/d/1w6m9h8FZNUbWVFA-4QIve5qZkGm2oBx-/view
<hr style="background-color:navy; align:left; height:2pt">

<h2 style="color:navy">Предобработка данных</h2>

### Загрузка библиотек и файлов

In [ ]:
# если не загружена какая-либо из библиотек:
# ! pip install folium
# ! pip install ipywidgets
# ! pip install geopandas
# ! pip install shapely
# ! pip install pyogrio
# ! pip install folium matplotlib mapclassify

In [ ]:
# импорт библиотек
import pandas as pd

# библиотеки для визуализации
import seaborn as sns

import matplotlib as mpl
from matplotlib import pyplot as plt    

In [ ]:
# библиотеки для чтения и создания географических координат
import json
from shapely import geometry

# библиотеки для анализа и визуализации геоданных
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd

# библиотека для интерактивной карты
import ipywidgets as widgets

In [ ]:
# загрузка датасета с пунктами питания
try: 
    rest = pd.read_csv('/datasets/moscow_places.csv')
except:
    try:
        rest = pd.read_csv('moscow_places.csv')
    except: 
        rest = pd.read_csv('https://code.s3.yandex.net/datasets/moscow_places.csv')

In [ ]:
# загрузка файла с геоданными административных округов
try:
    ao_json = json.load(open('/datasets/admin_level_geomap.geojson', 'r', encoding='utf8'))
except:
    try:
        ao_json = json.load(open('admin_level_geomap.geojson', 'r', encoding='utf8'))
    except: 
        value = 'https://code.s3.yandex.net/data-analyst/admin_level_geomap.geojson'
        ao_json = json.load(open(value, 'r', encoding='utf8'))        

In [ ]:
# загрузка датасета с геоданными муниципальных образований (районов)
try:
    value = 'https://drive.google.com/uc?export=download&id=1P6ById-iwqa-5iliDeJ1D6r0ifHgDi7_'
    mo_df = gpd.read_file(value)    
except:
    try: 
        mo_df = gpd.read_file('http://gis-lab.info/data/mos-adm/mo.geojson')
    except:
        mo_df = gpd.read_file('mo.geojson')

In [ ]:
# загрузка датасета с площадями и населением районов
try:
    mo_area = pd.read_csv('mo_area.csv', sep='\t', encoding='utf8')
except:
    value = 'https://drive.google.com/uc?export=download&id=1HdFP39flaJ6CbyaRM8Ci8OpJ2ZG4mPi-'
    mo_area = pd.read_csv(value, sep='\t', encoding='utf8')

Два файла с данными по муниципальным округам можно также скачать по ссылкам:
1. [геоданные](https://disk.yandex.ru/d/JyDAh6Nwmizn7A "скачать файл") или [gis-lab.info](https://gis-lab.info/qa/moscow-atd.html) 
2. [информация о площади и населении](https://disk.yandex.ru/d/sWj8c1RUGnUY0A) создана на основе таблицы из 
[Википедии](https://ru.wikipedia.org/wiki/Районы_и_поселения_Москвы).

### Чтение данных

#### Таблица rest

In [ ]:
# Количество записей, типы данных в столбцах и наличие дубликатов
rest.info()

In [ ]:
# переименуем несколько столбцов для краткости
rest = rest.rename(columns={
             'avg_bill': 'bill_info',             
             'middle_coffee_cup': 'coffee'})
rest = rest.rename(columns={
             'avg_bill': 'bill_info',             
             'middle_avg_bill': 'avg_bill'})

# выведем произвольные строки таблицы
display(rest.sample(5))

#### Таблица mo_df

In [ ]:
# информация о датафрейме с данными о районах Москвы
mo_df.info()

In [ ]:
# оставим только нужные столбцы
mo_df = mo_df[['NAME', 'ABBREV_AO', 'geometry']]

# переименуем столбцы
mo_df.columns = ['area', 'ward', 'geometry']
# посмотрим как выглядят произвольные строки
display(mo_df.sample(3))

#### Таблица mo_area

In [ ]:
# посмотрим как выглядят произвольные строки
display(mo_area.sample(3))

#### Определение переменных

Зададим необходимые для визуализации цветовые палитры и географические координаты.

In [ ]:
# список цветов для категорий
cat_color = ['#f29e00', '#fdc800', '#7794a3', '#2b8524', 
             '#16537d', '#cf2c2c', '#61cc64', '#d48281']

# список из 11 цветов для других графиков
color_list = ['#cf2c2c', '#16537d', '#d48281', '#7794a3', '#f29e00', '#61cc64', 
              '#2b8524', '#e83873', '#3374c0', '#fdc800', '#4e9dfc']

# географические координаты Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

#### Описание данных

Датафрейм `rest` содержит следующую информацию:
- `name` — название;
- `category` — категория («кафе», «пиццерия» и т.п.);
- `address` — адрес;
- `district` — административный район;
- `hours` — дни и часы работы;
- `lat` — географическая широта;
- `lng` — географическая долгота;
- `rating` — рейтинг по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);
- `price` — категория цен («средние», «ниже среднего» и т.д.);
- `bill_info` — средняя стоимость заказа в виде диапазона («Средний счёт: 1000–1500 ₽» и т.д.);
- `avg_bill` — средний чек (руб., рассчитан по значениям «Средний счёт» из avg_bill);
- `coffee` — средняя цена чашки кофе (руб., по «Цене одной чашки капучино» из avg_bill);
- `chain` — признак сети (для маленьких сетей могут встречаться ошибки);
- `seats` — количество посадочных мест.

Гео-датафрейм `mo_df` состоит из полей:
- `area` — название муниципального округа (района);
- `ward` — сокращённое название административного округа;
- `geometry` — географические координаты в виде полигона для муниципального округа.

В таблице `mo_area`:
- `area` — название муниципального округа
- `ward` — сокращённое название административного округа
- `square` — площадь муниципального округа (района)
- `population` — население

Два последний датафрейма объединить пока не получится, т.к. гео-датафрейм содержит записи о старых и новых административныз округах. А mo_area - только старые. 

### Пропущенные значения

#### Подсчёт пропусков

Посчитаем сколько пропущено значений в столбцах

In [ ]:
df_temp = rest.isna().sum().to_frame()
df_temp.columns = ['total']
df_temp['mean'] = round(rest.isna().mean(), 4)
df_temp = df_temp.query('total > 0')
display(df_temp.style.background_gradient('Blues').format('{:0.2%}', subset='mean'))

#### Пропуски в текстовых полях

Можем заменить пропуски на «**не указано**» в столбцах:
- `hours` - время работы;
- `avg_bill` -описание диапазонов среднего чека;
- `price` - уровень цен.

In [ ]:
rest[['hours', 'bill_info', 'price']] = rest[['hours', 'bill_info', 'price']].fillna('не указано')

#### Пропуски в полях с ценами

В столбцах со средним чеком и ценой за чашку кофе очень много проущенных значений. Эти суммы были рассчитаны автоматически по информации из bill_info, в котором нет информации более чем в половине записей. Посмотрим, есть ли нулевые значения в столбцах со средним чеком и ценой за чашку кофе.

In [ ]:
display(rest[['avg_bill', 'coffee']].min().to_frame())
# найдём заведения, где кормят бесплатно
rest.query('avg_bill==0')

Величину среднего чека в Кофемании на Новом Арбате явно посчитали некорректно, т.к. была указана только нижняя граница этого значения. Других нулевых значений нет в этих двух столбцах нет, поэтому можем заменить пропуски нулями. Главное, не забыть исключить их при анализе размера среднего чека. Кроме того, переведём эти поля в целочисленный формат, чтобы не считать до копейки. 

In [ ]:
rest[['avg_bill', 'coffee']] = rest[['avg_bill', 'coffee']].fillna(0).astype(int)

#### Пропуски в количестве мест

In [ ]:
value = rest.query('seats==0').seats.count()
print(f'Количество заведений без сидячих мест: \033[1m{value}\033[0m')

В данном случае сложно предположить чем можно заменить пропущенные значения. Всё же удобно, когда количество принимает целое значение. До тех пор, пока в столбце содержатся пропущенные значения, перевести его в целочисленный формат не получится. Заменим пропущенные значения на «-1», но обязательно будем исключать их при любых подсчётах в столбце `seats`. 

In [ ]:
rest.seats = rest.seats.fillna(-1).astype(int)

### Изучение содержимого датафрейма

#### Названия заведений

Так как в нашем проекте не принципиальны названия каждой отдельной точки, переведём названия в нижний регистр

In [ ]:
rest.name = rest.name.str.lower()

#### Категории

Убедимся, что в значениях столбца `category` нет ошибок

In [ ]:
display(rest.category.value_counts().to_frame())

- Заменим «бар,паб» на просто «**бар**». 
- Заменим «быстрое питание» на «**фастфуд**», чтобы сократить подписи на графиках.
- И сохраним список категорий в `cat_list`.

In [ ]:
rest.category = rest.category.replace('бар,паб', 'бар')
rest.category = rest.category.replace('быстрое питание', 'фастфуд')
cat_list = sorted(rest.category.unique())

#### Количество мест
Посмотрим на самые большие значения этого столбца

In [ ]:
display(rest.query('seats==seats.max()').head(3))

Мы вывели только 3 строки, но их гораздо больше и все заведения находятся по разным адресам на проспекте Вернадского. Такое ощущение, что для всех этих строк посчитали суммарные значения по всему проспекту. К сожалению, не понятно что делать с этими значениями. Скорее всего их придется просто обнулить. 

In [ ]:
rest.seats = rest.seats.replace(1288, 0)

In [ ]:
display(rest.query('seats==seats.max()'))

Какая-то слишком уж большая столовая. Разделим эту цифру на 10. Остальные значения оставим в первоначальном виде. Просто будем иметь ввиду, что среди них имеются выбросы.

In [ ]:
rest.loc[4231, 'seats'] = 120

#### Сетевые и несетевые заведения

Метку того, что запись относится к какой-либо сети переведём для удобства в логический тип.

In [ ]:
rest.chain = rest.chain.astype('bool')

print('\033[1mСетевые заведения, представленые по названию ', end='')
print('в единственном числе:\033[0m')
list_tmp = rest.query('chain').name.value_counts().to_frame().query('name==1').index
print(*list_tmp, sep=', ')

Что касается выведенного выше списка, возможно, что некоторые сети используют для своих точек разные названия или в некоторых названиях есть ошибки или уточнения. Как бы ни было, не понятно, к какой именно сети относятся указанные точки. Исключим их из сетевых.

In [ ]:
rest.chain = rest.chain.mask(rest.name.isin(list_tmp), False)

#### Время работы 

Добавим метку для круглосуточных заведений. 

In [ ]:
# проконтролируем, что все значения в нижнем регистре
rest.hours = rest.hours.str.lower()

rest['is_24/7'] = (rest['hours'].str.contains("ежедневно, круглосуточно"))

#### Уровень цен

In [ ]:
display(sorted(rest.price.unique()))

С категориями по уровню цен всё в порядке, но такую информацию неудобно будет сортировать в случае необходимости. Поэтому:
- создадим список `prices_list` с уровнями цен, отсортированными в логичном порядке;
- продублируем эти значения цифровым полем 'prices' в таблице.

In [ ]:
prices_list = ['не указано', 'низкие', 'средние', 'выше среднего', 'высокие']
rest['prices'] = rest.price.replace(prices_list, range(0,5))          

#### Средние суммы чека

Проверим есть ли аномальные значения в ценах

In [ ]:
df_tmp = rest.query('avg_bill>0').groupby('category').avg_bill.describe()
df_tmp = df_tmp[['count', 'min', '50%', 'max']].astype(int).T
display(df_tmp)

Посмотрим нет ли элементарной ошибки в случае среднего чека на 35 тысяч рублей. Выведем все заведения, где эта сумма выше, например 7 тысяч.

In [ ]:
rest.query('avg_bill>7000')

1. Чойхону проверим по адресу.
2. Цены в Белом Кролике действительно высокие.
3. Возможно, это заведение для праздников, которое работает только 2 раза в неделю. Либо это тоже ошибка. Много данных мы не потеряем, если исключим эту строку из данных.
4. Казалось бы, что в суммы для то ли кафе то ли ресторана на Каширском шоссе закрались ошибки. Но, к моему огромному сожалению, я знаю что там находится. Это морг при онкоцентре. А кафе и ресторан используется для организации прощальных мероприятий. Вряд ли клиент планирует октрыть подобное заведение, судя по его запросам. И в качестве обычного заведения это место тоже не подходит. Просто исключим эту строку из данных.

In [ ]:
rest.query('address.str.contains("Дмитровское шоссе, 95")')

Больше похоже на ошибочную строку. Чойхону скорее всего переименовали в Тандыр. А так как уровень цен сомнителен, удалим вторую из этих двух строк. И проверим цены в пунктах быстрого питания.

In [ ]:
rest = rest.query('avg_bill<10000')
rest.query('category=="фастфуд" & avg_bill>2000')

1. Исправим цены для "Крошки картошки", уменьшив в 10 раз. Это ошибка.
2. phobo - малоизвестная сеть, но средняя цена во всех заведениях этой сети 500 рублей. Исправим и эту строку. 
3. Неизвестно что за Саят-Нова. Но в Капотне не очень много заведений. Возможно, многие заказывают еду сразу на компанию. Поэтому и цена среднего чека получается высокой. 

Остальные аномалии не будем исправлять, чтобы не тратить время. Просто будем иметь ввиду, что они присутствуют в наших данных.

In [ ]:
rest.loc[2795,'avg_bill'] = 550
rest.loc[4269, 'avg_bill'] = 500

#### Цена за чашку кофе

Аналогично предыдущему пункту, посмотрим есть ли аномалии в этом столбце.

In [ ]:
rest.query('coffee>400')

Пожалуй, это единственная здесь аномалия - в информации о среднем чеке неверно указали максимальное значение. По всем остальным Шоколадницам средняя цена за чашку кофе стандартно составляет 256 руб. 

In [ ]:
rest.loc[2859, 'coffee'] = 256

### Работа с адресами

#### Округ

Проверим, что названия округов в порядке.

In [ ]:
display(sorted(rest.district.unique()))

- Сохраним в таблицу `ward_df` длинные и сокращённые названия округов.
- Добавим столбец `ward` с общепринятыми сокращениями для названий округов, чтобы удобнее было использовать на графиках.

In [ ]:
%%capture --no-display
# создадим столбец с индексами и значениями равными названиям округов
ward_df = pd.Series(data = sorted(rest.district.unique()),
                    index = sorted(rest.district.unique()))
# значения столбца сделаем сокращенями (питон немного ворчит, но пока делает)
ward_df = ward_df.str.title()
ward_df = ward_df.str.replace(r'[а-я -]', '') 

# добавим столбец с сокращениями в датафреймы rest и mo_df
rest['ward'] = rest.district.apply(lambda x: ward_df[x])

# приведём таблицу ward_df к обычному виду
ward_df = ward_df.reset_index()

# добавим названия столбцов
ward_df.columns = ['district', 'ward']

#### Город и округ в строке с адресом

Посмотрим, не затесались ли в адрес другие населённые пункты кроме Москвы. 

In [ ]:
display(rest.address.str.split(',', expand=True)[0].value_counts().to_frame())

Сначала исключим слово "Москва" из всех адресов, затем посмотрим, не содержит ли этот столбец название округа.

In [ ]:
rest.address = rest.address.str.replace('Москва, ', '')
display(rest.query('address.str.contains("округ")').sample(3))

Информация об административных округах уже есть в поле district. Уберём округа из адреса.

In [ ]:
rest.address = rest.address.mask(rest.address.str.contains("округ"), 
                  rest.address.str.split(', ', n=1, expand=True)[1])

#### Район в строке адреса

Чтобы "извлечь" улицу из адреса, проверим также наличие "района" в названии

In [ ]:
display(rest.query('address.str.contains("район")').sample(3))

- Есть строки где в качестве адреса просто указано название района. Зачастую это небольшие кафе или кофейни на большом открытом пространстве, не имеющем более точного названия. Например, "район Строгино". 
- Кроме того, в этом списке есть точные адреса, в которых указан район города, а затем название улицы. В таком случае название района можно убрать из адреса. 

In [ ]:
# чтобы не удлинять вывод, следующая строчка закомменитрована
# "районы" из адреса проанализированы "вручную", их не очень много
'''
display(sorted(rest.query('address.str.contains("район") & address.str.contains(",")')
        .address.unique()))
''';

Произведём несколько замен, а затем исключим название района из адреса.

In [ ]:
rest.address = rest.address.replace('2-й квартал Капотня', 'район Капотня')
rest.address = rest.address.replace('2-й квартал Капотни', 'район Капотня')
rest.address = rest.address.replace('район Капотня', 'Капотня')
rest.address = rest.address.replace('территория парка', 'парк')

rest.address = rest.address.mask(
               rest.address.str.contains("район") & rest.address.str.contains(","), 
               rest.address.str.split(', ', n=1, expand=True)[1])

#### Улицы
Создадим столбец `street` с названиями улиц, а столбец с адресами переведём в нижний регистр, чтобы было проще искать дубликаты.

In [ ]:
rest['street'] = rest.address.str.split(',', expand=True)[0]
rest.address = rest.address.str.lower()

In [ ]:
# оставим только центральные районы
mo_df = mo_df.query('ward.isin(@ward_df.ward)')

#### Районы

В проекте предлагается анализировать данные по улицам. Считаю это нецелесообразным, так как невозможно сравнить МКАД или Варшавское шоссе с небольшим переулком в центре города. Проще провести анализ по районам и муниципальным образованиям. Во-первых, районов гораздо меньше, чем улиц. Во-вторых, районы больше похожи по своим размерам.

По полигонам из mo_df определим район для каждого заведения и запишем в столбцец `area`. Для этого определим функцию `mo_find`, которая по географическим координатам определяет название района.

In [ ]:
# функция определения муниципального района
def mo_find (lng, lat):
    point = geometry.Point(lng, lat)
    return mo_df[point.within(mo_df.geometry)].values[0][0]

# добавим районы в датафрейм rest (работает медленно!)
rest['area'] = rest[['lng', 'lat']].apply(lambda x: mo_find(x[0], x[1]), axis=1)

#### Соответствие районов и округов

Мы включили в таблицу данные о районах из другого источника. Поэтому один и тот же район для пограничных адресов мог попасть не в свой округ. Проверим это.

In [ ]:
rest.groupby('area').ward.nunique().to_frame().query('ward>1')

Найдём к какому округу относится этот район и в каких записях есть ошибки и исправим название округа по этим адресам.

In [ ]:
display(mo_df.query('area=="Донской"'))
display(rest.query('area=="Донской" & ward!="ЮАО"'))

Исправим две строки, чтобы не возникло путаницы при анализе.

In [ ]:
rest.district = rest.district.mask((rest.area=="Донской") & (rest.ward!="ЮАО"), 
                          'Южный административный округ')
rest.ward = rest.ward.mask((rest.area=="Донской") & (rest.ward!="ЮАО"), 'ЮАО')

Оставим в таблицах mo_df и mo_area ровно столько районов, сколько содержит датайрем rest. Кроме того, заменим небольшой кусок полигона из Кунцева, куда попала область Новой Москвы.

In [ ]:
# оставим только центральные районы
list_tmp = rest.area.unique()
mo_df = mo_df.query('area.isin(@list_tmp)')
mo_area = mo_area.query('area.isin(@list_tmp)')

# определим номер строки Кунцево и полигон 
value = mo_df.query('area=="Кунцево"').index
M = mo_df.query('area=="Кунцево"').geometry.values

mo_df.loc[value, 'geometry'] = geometry.MultiPolygon([P for P in M[0].geoms if P.bounds[2]>37.2])

# проверка
# print(len(list_tmp)==len(mo_df), len(list_tmp)==len(mo_area))

### Дубликаты

#### Поиск и удаление дубликатов

In [ ]:
value = rest.duplicated().sum()
print(f'Количество явных дубликатов: \033[1m{value}\033[0m.')

value = rest[['name', 'address']].duplicated().sum()
print(f'Совпадает название и адрес в \033[1m{value}\033[0m случаях.')

Посмотрим в каких случаях по одному и тому же адресу оказались одинаковые заведения. 

In [ ]:
rest[rest[['name', 'address']].duplicated(keep=False)]

- кафе на Ангарских прудах просто сменило часы работы;
- ресторан "more poke" поменял часы работы и добавил (или убрал) несколько посадочных мест;
- в ресторане "раковарня" есть ещё и бар. Незначительно отличаются часы работы - в пятницу и в субботу сложно выгнать посетителей из бара раньше часа ночи. Количество мест совпадает, это явно одно и то же заведение. Оставим ресторан, так как в ресторане почти всегда можно заказать спиртное, а в баре полноценно поесть можно не всегда;
- булочная и кафе с большим количеством мест скорее всего находятся где-нибудь в торговом центре, а булочная использует столики фуд-корта для своих клиентов. 

Во всех случаях можем удалить нижнюю строку.

In [ ]:
rest = rest[~rest[['name', 'address']].duplicated(keep='first')]

#### Фуд-корты

При изучении данных было замечено что для кафе и пунктов быстрого питания в фуд-кортах количество посадочных мест указывается одинаковым. Если выделить такие точки на карте, то мы сможем не только проанализировать количество фуд-кортов, но и исключить многократный подсчёт одних и тех же посадочных мест.
Посмотрим как выглядят такие записи на примере найденного выше дубликата кафе и булочной по адресу.

In [ ]:
display(rest.query('address.str.contains("ярцевская улица, 19")')[[
        'name', 'address', 'seats']].head(5))

Это не все строчки, их почти в 4 раза больше. И у всех одно и то же число в поле seats. Понятно, что для рядового кафе это огромное количество мест. А если сложить эти цифры многократно, то это  исказит реальные суммы. 

Поступим следующим образом:
- сначала создадим список адресов `court_addr`, по которым зарегистрировано, например 3 или больше заведений, количество мест в которых, скажем больше 8;
- найдём по каким из этих адресов имеется одинаковое количество мест; 
- добавим в таблицу столбец с меткой `food_court`. 

Учтём, что даже если в этот список попадут кафе и рестораны находящиеся в одном и том же здании, но не указавшие количество мест и не относящиеся к фуд-кортам, то мы всё равно не сможем посчитать количество посадочных мест в них (в наших данных оно указано как -1). Поэтому это не повлияет на результат. 

In [ ]:
court_addr = rest.query('seats>8').address.value_counts().to_frame()
court_addr = court_addr.query('address>2').index

value = len(court_addr)
print('Количество адресов, по которым зарегистрировано 3 и более ', end = '')
print(f'заведений: \033[1m{value}\033[0m.')

value = len(rest.query('address.isin(@court_addr)'))
print(f'По этим адресам находятся \033[1m{value:,}'.replace(',', ' '), end = '')
print('\033[0m точек общественного питания.')

Убедимся, что по каждому из выбранных адресов количество посадочных мест во всех заведениях совпадает:

In [ ]:
display(rest.query('address.isin(@court_addr)').groupby('address')
        .seats.nunique().value_counts(). to_frame())

Укажем, что все эти заведения находятся в фуд-кортах.

In [ ]:
rest['food_court'] = rest.address.isin(court_addr)

Чтобы было проще рассчитывать количество мест в таких заведениях, добавим в наш датафрейм поле `seats_avg`, в котором для выбранных адресов поставим количество мест, усреднив значения. Для остальных пунктов питания значение оставим прежним. 

In [ ]:
# для выбранных адресов рассчитаем количество мест и заведений по каждому адресу
court_df = rest.query('food_court').groupby('address').seats.first().to_frame()
court_df['name_cnt'] = rest.query('food_court').groupby('address').name.count()
# найдём среднее значение и приведём к целому числу
court_df['seats_avg'] = round(court_df.seats / court_df.name_cnt).astype(int)

# добавим новое поле в датафрейм
rest['seats_avg'] = rest.address.apply(
    lambda x: court_df.seats_avg[x] if x in court_addr else 0)
rest.seats_avg = rest.seats_avg.mask(~rest.food_court, rest.seats)

### Итоги предобработки данных

Чтобы было удобно работать с "широкой" таблицей" расставим столбцы в смысловом порядке.

In [ ]:
rest = rest[['name', 'category', 
             # адресные данные
             'district', 'ward', 'area', 'street', 
             'address', 'lat', 'lng', 
             # рейтинг и количество мест и т.д.
             'chain', 'rating', 'seats', 'seats_avg', 'food_court', 
             # время работы
             'hours', 'is_24/7', 
             # цены
             'prices', 'price', 'bill_info', 'avg_bill', 'coffee']]

С учётом новых столбцов `rest` содержит следующую информацию:

**Основная информация**
- `name` — название;
- `category` — категория, список сохранён в `cat_list`;

**Местоположение**
- `district` — административный район, список сохранён в одноимённом столбце таблицы `ward_df`;
- `ward` — сокращённое название района, дублируется в столбце `ward_df`;
- `area` — муниципальный район, список соответствий с округами есть в `mo_df`;
- `street` — улица;
- `address` — сокращённый адрес;
- `lat` — географическая широта;
- `lng` — географическая долгота;

**Признаки, рейтинг, количество мест**
- `chain` — признак сети;
- `rating` — рейтинг по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);
- `seats` — количество посадочных мест в исходных данных;
- `seats_avg` — количество мест с учётом общих столиков в фуд-кортах;
- `food_court` — признак того, что пункт питания находится в фуд-корте;

**Время работы**
- `hours` — дни и часы работы;
- `is_24/7` — признак того, что заведение работает круглосуточно;

**Цены**
- `prices` — номер категории по уровню цены (от 0 - "уровень не указан" до 4 - "высокие цены");
- `price` — категория цен из списка `prices_list`;
- `bill_info` — средняя стоимость заказа в виде диапазона;
- `avg_bill` — средний чек;
- `coffee` — средняя цена чашки кофе.

<h2 style="color:navy">Анализ данных</h2>

### Общие расчёты

#### Определение признаков для изучения

В ходе анализа будем изучать:
1. **Количественные признаки**: количество заведений, количество посадочных мест, рейтинг, величину среднего чека, среднюю цену чашки кофе.
2. **Основные категориальные признаки**: категории заведения, административные округа, муниципальные районы, улицы.
3. **Остальные категориальные признаки**: сетевые точки, круглосуточные заведения, расположение в фудкорте, уровень цен.

В силу ограничения времени, возможно не все эти данные будут исследованы полностью. Постараемся выделить наиболее важные. Сначала найдём общее количество заведений и посадочных мест в них. Сохраним эти значения для дальнейших расчётов.

#### Количество заведений

In [ ]:
name_cnt = len(rest)
print('Количество заведений общественного питания в Москве: ', end ='')
print(f'\033[1m{name_cnt:,}\033[0m.'.replace(',', ' '))
print()

name_seats = len(rest.query('seats>0'))
print('Информация о количестве посадочных мест есть для ', end ='')
print(f'\033[1m{name_seats:,}\033[0m заведений.'.replace(',', ' '))

court_cnt = len(rest.query('food_court'))
value = court_cnt / name_seats
print(f'- \033[1m{court_cnt:,} ({value: 0.1%})\033[0m '.replace(',', ' '), end='')
print('из них предположительно расположены в фуд-кортах.')

#### Количество мест в заведениях

In [ ]:
seats_avg_cnt = rest.query('seats>0').seats_avg.sum()
value = rest.query('seats>0').seats.sum()

print('Общее количество мест в заведениях, где это число указано:')
print(f'- \033[1m{value:,}\033[0m - '.replace(',', ' '), end ='')
print('если считать по всей таблице без учёта фуд-кортов;')
print(f'- \033[1m{seats_avg_cnt:,}\033[0m - '.replace(',', ' '), end ='')
print('по усреднённым значениям для мест "общего пользования".')
print()

court_seats = rest.query('food_court').seats_avg.sum()
value = court_seats / seats_avg_cnt
print('Приблизительное количество общих посадочных мест в фуд-кортах: ', end='')
print(f'\033[1m{court_seats:,}\033[0m,'.replace(',', ' '))
print(f'- или \033[1m{value: 0.1%}\033[0m от общего количества мест по усреднённым данным.')

Интересная получилась статистика. Заведения, которые используют общие столики, видимо сильно экономят на количестве мест. Будем иметь ввиду, что расчёты количества мест в сумме неточны априори, потому что:
- почти для половины заведений число посадочных мест неизвестно;
- деление на фуд-корты производилось условно и может быть некорректным;
- небольшие погрешности могли возникнуть при округлении усреднённых значений.

Т.е. не следует сильно полагаться на этот признак при выборе условий для открытия новой точки.

### Сравнение категорий

#### Количество заведений по категориям

Создадим датафрейм `cat_df` где соберём различные количественные признаки для каждой категории. Визуализируем с помощью круговой диаграммы, убрав самые малочисленные категории отдельную группу.

In [ ]:
cat_df = rest.groupby('category').name.count().to_frame()
cat_df.columns = ['name_cnt']
cat_df['name_pct'] = cat_df.name_cnt / name_cnt

# добавим столбец соответствия цветов категориям
cat_df['color'] = cat_color
# отсортируем
df_tmp = cat_df.sort_values(by='name_cnt', ascending=False).reset_index()
# расчитаем значение для строк ниже четвёртой
value = df_tmp.query('index>3').name_cnt.sum()
# заменим нижние строки на "другие"
df_tmp.category = df_tmp.category.mask(df_tmp.index>4, 'другие')
df_tmp.name_cnt = df_tmp.name_cnt.mask(df_tmp.index>4,  value)
# оставим только нужное
colors_tmp = df_tmp.color
df_tmp = df_tmp.set_index('category').name_cnt.head(6)

# размер шрифта подписей на графиках
mpl.rc('font', size=14)
# круговая диаграм
df_tmp.plot.pie(subplots=True, autopct='%1.1f%%', 
        explode=[0.05,0.05,0.05,0.05,0.05,0.2], 
        pctdistance = 0.7,
        colors = colors_tmp, ylabel='', 
        figsize=(5,5), legend=False)
plt.title('Количество заведений по категориям', size=14)

plt.show();

#### Количество мест по категориям

Найдем сколько мест в заведениях по разным категориям.

In [ ]:
cat_df['seats_cnt'] = rest.query('seats>0').groupby('category').seats_avg.sum()
cat_df['seats_pct'] = cat_df.seats_cnt / seats_avg_cnt
cat_df['seats_mean'] = round(cat_df.seats_cnt / cat_df.name_cnt, 2)

display(cat_df[cat_df.columns.drop('color')]
        .sort_values(by='seats_cnt', ascending=False)
        .style.background_gradient('Blues', subset='seats_cnt')
        .format({'name_pct': '{:0.2%}',
                 'seats_pct': '{:0.2%}',
                 'seats_mean': '{:0.2f}'}))

- Количество мест в заведениях по категориям распределилось практически также, как и количество самих заведений. С единственной разницей - поменялись местами кафе и рестораны. Но это очевидно, так как количество мест в ресторанах как правило больше, чем в кафе. 

- В последнем столбце таблицы мы рассчитали среднее количество мест в заведении. В процессе изучения данных было замечено, что если выбрать в качестве адресов фуд-кортов точки, где расположились 4 и более заведения (мы взяли 3), то среднее количество мест получалось очень завышенным, что было особенно заметно на примере булочных.  

- Ещё раз убеждаемся, что не стоит учитывать количество мест в заведениях при нашем анализе из-за низкой точности таких вычислений и большого числа пропущенных значений. 

- Смысла строить ещё одну круговую диаграмму, почти сопвадающую с предыдущей также нет.

In [ ]:
cat_df = cat_df[['name_cnt', 'color']]

#### Сетевые заведения по категориям

In [ ]:
cat_df['chain_cnt'] = rest.groupby('category').chain.sum()
cat_df['chain_mean'] = cat_df.chain_cnt / cat_df.name_cnt

# отсортируем по количеству заведений
df_tmp = cat_df.sort_values(by='name_cnt')

# построим диаграмму по количеству заведений
fig = df_tmp.name_cnt.plot.barh(figsize=(10,5), xlim=(0,2500), width=.8, 
    xlabel='', color=df_tmp['color'], alpha=0.5)

# на тех же осях добавим доли сетевых точек
df_tmp.chain_cnt.plot.barh(width=.8, xlabel='', color=df_tmp['color'], ax=fig)

# подпишем количество заведений
i = -.2
for row in df_tmp['name_cnt']:
    plt.text(row+10, i, row, horizontalalignment='left')
    i+=1

# подпишем проценты долей
df_tmp = df_tmp = round(df_tmp['chain_mean']*100,1).astype('string')+'%'
i = -.2
for row in df_tmp:
    plt.text(-500, i, row, horizontalalignment='right')
    i+=1
plt.text(-750, -1.5, 'доля сетевых', horizontalalignment='left')

# скроем верхнюю и правую линии на осях
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)

plt.title('Количество и доли сетевых заведений по категориям', size=14)
plt.xlabel('количество заведений')
plt.show();

- Больше всего сетевых заведений среди булочных и пиццерий - более половины из всех имеющихся. Объяснить можно тем, что для таких заведений полезно и удобно иметь свою пекарню. А если есть пекарня, то можно готовить хлеб и пиццы сразу для нескольких точек. 
- На третье место по популярности сетевых заведений вышли кофейни. Скорее всего поставлять вкусные и разнообразные сорта кофе тоже удобнее в несколько кофеен сразу. 
- Реже всего сетевыми бывают бары. Чаще это "штучные" заведения со своей атмосферой. 

#### Другие признаки по категориям

Рассчитаем количество круглосуточных заведений, точек в фуд-кортах, средний рейтинг и средний уровень цен для разных категорий. Посмотрим что получилось, чтобы не упустить каких-либо интересных особенностей. 

In [ ]:
cat_df['24/7_cnt'] = rest.groupby('category')['is_24/7'].sum()
cat_df['24/7_mean'] = cat_df['24/7_cnt'] / cat_df.name_cnt

cat_df['fc_cnt'] = rest.groupby('category').food_court.sum()
cat_df['fc_mean'] = cat_df.fc_cnt / cat_df.name_cnt

cat_df['rating_avg'] = rest.groupby('category').rating.mean()
cat_df['price_level'] = rest.query('prices>0').groupby('category').prices.mean().astype(int)

display(cat_df[cat_df.columns.drop('color')]
        .sort_values(by='name_cnt', ascending=False)
        .style.background_gradient('Blues', subset=['chain_mean', '24/7_mean', 'fc_mean'])
        .format({'chain_mean': '{:0.2%}', '24/7_mean': '{:0.2%}',
                 'fc_mean': '{:0.2%}', 'rating_avg': '{:0.1f}'}))

**Круглосуточные заведения**
- Количество тех, кто работает 24/7 в категории пунктов быстрого питания гораздо выше, чем в других категориях. Логично, что ночью меньше желающих сидеть за столиками в ресторане или кафе. 
- Между тем более 10% всех кафе работают круглосуточно. 
- Доля круглосуточных заведений среди кофеен одна из самых низких. Тоже не удивительно, большинство клиентов предпочитают пить кофе в первой половине дня. 

**Расположение в фуд-кортах**
- На первом месте опять заведения быстрого питания. Фуд-корты чаще всего располагаются в больших торговых рах. Многие посетители всего лишь перекусывают в перерыве между посещениями магазинов, кинотеатра и т.п.  
- Интересно, что рестораны и даже бары "не брезгуют" использовать общие столики. Но, как мы выяснили выше, информация о количестве мест в наших данных весьма сомнительного качества. 
- Реже всего в фуд-корты попадают столовые. 

**Рейтинг и уровень цен**
- Средние значения этих признаков в разных категориях практически не отличаются. За исключением очевидного факта, что посещать бары и рестораны - дороже, чем всё остальное.
- Самым дешевым местом, чтобы поесть стали столовые.

**Столовые**
- Как было замечено выше, столовых и булочных всего по 3 с небоьшим процента от общего количества всех заведений. Здесь интересно было бы уточнить когда заведение принято называть столовой. Либо нужно большое количество столиков, либо нужен свой цех по приготовлению полноценного набора блюд, как в ресторане, но при этом в более быстром и дешевом варианте. Вообще столовые - довольно редкое явление. Однако стоит учесть, что столовые есть в каждой школе, в детских садах, на предприятиях и т.п. То есть в закрытых для общего посещения местах. Поэтому говорить о нехватке столовых в городе не стоит, не изучив другие данные.

### Сетевые заведения

#### Количество и топ-15 сетевых заведений

In [ ]:
value = len(rest.query('chain'))
print('Количество сетевых заведений ', end='')
print(f'\033[1m{value:,}\033[0m, '.replace(',', ' '), end='')

value /= name_cnt
print(f'что составляет\033[1m{value: 0.1%}\033[0m от общего количества.')

Найдём топ-15 сетей с наибольшим количеством заведений. Сохраним эту иформацию в таблице `chain_df`. Найдём, не осталось ли пунктов питания с такими же названиями, но отмеченными как сетевые.

In [ ]:
chain_df = rest.query('chain').name.value_counts().to_frame().head(15)
print('\033[1mТоп-15 сетевых заведений:\033[0m')
print(*chain_df.index, sep=', ')
rest.query('name.isin(@chain_df.index) & ~chain')

Вряд ли кто-то просто так станет использовать чужие названия. Исправим информацию в этих двух строках.
Затем переименуем длинное название "кулинарная лавка братьев Караваевых" для удобства вывода в таблицах и графиках. 
И, наконец, рассчитаем основые количественные показатели для сетевых заведений.

In [ ]:
# поменяем значения в таблице rest
rest.name = rest.name.str.replace('кулинарная лавка братьев караваевых', 'братья караваевы')
rest.chain = rest.chain.mask((rest.name.isin(chain_df.index)) & (~rest.chain), True)

# снова рассчитаем топ-15
chain_df = rest.query('chain').name.value_counts().to_frame().head(15)
chain_df.columns = ['name_cnt']

# количество категорий для заведений одной сети
df_tmp = rest.query('name.isin(@chain_df.index)').groupby('name').category.nunique()
chain_df['cat_cnt'] = df_tmp[chain_df.index]

# список категорий
df_tmp = rest.query('name.isin(@chain_df.index)').groupby('name').category.unique()
chain_df['categories'] = df_tmp[chain_df.index]
chain_df.categories = chain_df.categories.apply(lambda x: sorted(x))

display(chain_df)

Все заведения из списка "на слуху". Му-му решила захватить все ниши. Оказывается, у них есть даже бар. 
Среди сетевых заведений больше всего кафе, ресторанов и кофеен. Посмотрим реальное количество каждой категории для топ-15 заведений, у которых больше одной категории.

In [ ]:
for cat in cat_list:
    list_tmp = []
    for tc in chain_df.index:
        list_tmp += [len(rest.query('category==@cat & name==@tc'))]
    chain_df[cat] = list_tmp
display(chain_df.query('cat_cnt>1'))    

За исключением Хинкальной и Му-Му, все заведения относятся по большей части к одной категории. Переименуем список категорий "вручную", уберём лишние столбцы и изучим много ли здесь круглосуточных точек.

In [ ]:
list_tmp = ['кофейня', 'пиццерия', 'пиццерия', 'кофейня', 'ресторан', 'кофейня', 'ресторан', 
            'кафе', 'кофейня', 'кафе', 'ресторан', 'кафе', 'кофейня', 'булочная', 'кафе']

chain_df = chain_df[['name_cnt']]
chain_df['category'] = list_tmp

df_tmp = rest.query('name.isin(@chain_df.index)').groupby('name')['is_24/7'].sum()
chain_df['24/7_cnt'] = df_tmp[chain_df.index]
chain_df['24/7_mean'] = chain_df['24/7_cnt'] / chain_df.name_cnt

display(chain_df
        .sort_values(by='name_cnt', ascending=False)
        .style.background_gradient('Blues', subset='24/7_mean')
        .format({'24/7_mean': '{:0.2%}'}))

In [ ]:
# отсортируем в нужном для графика порядке и оставим только два столбца
chain_df = chain_df.sort_values(by='name_cnt')[['name_cnt', '24/7_cnt']]
# вычтем из общего количества заведений число круглосуточных
chain_df['not_24'] = chain_df.name_cnt - chain_df['24/7_cnt']
# переименуем столбцы, чтобы проще писать
chain_df.columns = ['name_cnt', 'is_24', 'not_24']
# умножим на -1 количество круглосуточных
chain_df.is_24 *= -1

# названия заведений напишем с большой буквы как они были изначально
chain_df.index = chain_df.index.str.title()

In [ ]:
# построим столбчатую диаграмму сначала по не круглосуточным
ax = chain_df.not_24.plot.barh(figsize=(10,7), 
     xlim=(-33,120), ylim=(-10, 16), width=.8, color=color_list)

# на этих же осях добавим влево круглосуточные заведения
chain_df.is_24.plot.barh(ax=ax, width=.8, color=color_list)

# проведём две линии линию
ax.vlines(0, -.5, 14.5, color='white')
ax.hlines(-.5, -33, 40, color='black')

# подпишем всё
i = -.2
for row in chain_df.index:
    is_24 = chain_df.loc[row, 'is_24']
    not_24 = chain_df.loc[row, 'not_24']
    all_ = chain_df.loc[row, 'name_cnt']
    
    plt.text(2, i, not_24, horizontalalignment='left', color = 'white')
    if is_24 < 0: plt.text(-2, i, -is_24, horizontalalignment='right')
    plt.text(not_24-1, i, all_, horizontalalignment='right')  
    plt.text(-33, i, row, horizontalalignment='right', color=cat_color[4])
    i+=1

plt.text(-3, -1, 'круглосуточные', size=12, horizontalalignment='right')
plt.text(-.2, -1, '|   не круглосуточные', size=12, horizontalalignment='left')

# скроем оси
ax.axis('off') 
plt.title('Топ-15 сетей общественного питания', size=16)

plt.show();

Можно выделить всего лишь три сети - Чайхана, Шоколадница и Яндекс лавка, у которых довольно большая доля круглосуточных заведений. Большинство сетевых точек не работают 24/7.

#### Категории сетевых заведений

Построим такую же круговую диаграмму, как мы строили выше, но только для сетевых заведений. 

In [ ]:
# отсортируем
df_tmp = cat_df.sort_values(by='chain_cnt', ascending=False).reset_index()
# расчитаем значение для строк ниже четвёртой
value = df_tmp.query('index>3').chain_cnt.sum()
# заменим нижние строки на "другие"
df_tmp.category = df_tmp.category.mask(df_tmp.index>4, 'другие')
df_tmp.chain_cnt = df_tmp.chain_cnt.mask(df_tmp.index>4,  value)
# оставим только нужное
colors_tmp = df_tmp.color
df_tmp = df_tmp.set_index('category').chain_cnt.head(6)

# круговая диаграм
df_tmp.plot.pie(subplots=True, autopct='%1.1f%%', 
        explode=[0.05,0.05,0.05,0.05,0.05,0.2], 
        pctdistance = 0.7,
        colors = colors_tmp, ylabel='', 
        figsize=(5,5), legend=False)
plt.title('Количество сетевых заведений по категориям', size=14)

plt.show();

Три лидера по количеству остались прежнеми. Это рестораны, кафе и кофейни. Но доля кофеен, пиццерий и пунктов быстрого питания среди сетевых заведений выше, чем среди всех. 

### Кластеризация 

Визуализируем наш датайрем, чтобы можно было подробнее изучить плотность расположения заведений по районам города. 

In [ ]:
# создаём карту Москвы
m = folium.Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# функция cluster принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def cluster(row):
    folium.CircleMarker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
        radius=8, color='#7794a3', fill=True, fill_color='#16537d', fill_opacity=1
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
rest.apply(cluster, axis=1)

# выводим карту
m

Немного "прогулявшись" по этой карте, замечаем очевидный факт - плотность заведений в центре города в несколько раз превышает ситуацию по окраинам. 

### Изучение административных округов

#### Количество заведений в округах

In [ ]:
ward_df['name_cnt'] = rest.groupby('ward').name.count().values
ward_df['color'] = color_list[0:9] 
ward_df = ward_df.set_index('ward')
df_tmp = ward_df.sort_values(by='name_cnt')

# построим диаграмму по количеству заведений
fig = df_tmp['name_cnt'].plot.barh(figsize=(10,5), width=.8, xlabel='',
        color=df_tmp['color'], alpha=.8)

# подпишем количество заведений
i = -.2
for row in df_tmp['name_cnt']:
    plt.text(20, i, row, horizontalalignment='left')
    i+=1

# скроем верхнюю и правую линии на осях
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
    
plt.title('Количество заведений по административным округам', size=14)
plt.xlabel('количество заведений')
plt.show();

Центральный округ опередил всех в более чем в два раза. Очень мало заведений в СЗАО. Можно конечно использовать и эти данные, если не знать, что Северо-Западный округ самый маленький по площади среди всех. Т.е. количество заведений на целый округ в данном случае ни о чем нам не говорит. 

#### Категории заведений по округам

Посмотреть как распределились заведения разных категорий по административным округам нам поможет тепловая карта.

In [ ]:
# рассчитаем нужные данные и запишем в таблицу ward_df
for cat in cat_list:
    list_tmp = []
    for tc in ward_df.index:
        list_tmp += [len(rest.query('category==@cat & ward==@tc'))]
    ward_df[cat] = list_tmp

# построим тепловую карту    
plt.figure(figsize=(16, 7))
sns.heatmap(ward_df[ward_df.columns.drop(['district', 'name_cnt', 'color'])].T,
            cmap=sns.color_palette("Blues", as_cmap=True), annot=True, fmt='') 

plt.xlabel('')
plt.title('Распределение категорий точек питания по округам') 
plt.show();

Видно, что основными заведениями в Москве во всех округах являются кафе и чуть реже - рестораны. Логично, что больше всего таких заведений в центре города. Опять учитываем, что СЗАО - самый маленький по площади из всех округов.

#### Средний рейтинг заведений по округам

Найдём средний рейтинг заведений каждого округа.

In [ ]:
rating_df = rest.groupby('district', as_index=False).rating.mean()
display(rating_df.sort_values(by='rating', ascending=False)
        .style.background_gradient('Reds')
        .format('{:0.2f}', subset='rating'))

Средние рейтинги практически не отличаются друг от друга. ЦАО опять впереди. И опять не удивительно - многие хорошие рестораны и кафе в центре города работают не первый десяток лет и пользуются популярностью. 

In [ ]:
# создаём карту Москвы
m = folium.Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
folium.Choropleth(geo_data = ao_json, data = rating_df, columns = ['district', 'rating'],
    key_on = 'feature.name', fill_color = 'Reds', fill_opacity = 0.8,
    legend_name='Средний рейтинг заведений по округам').add_to(m)

# выводим карту
m

### Распределение заведений по районам

#### Количество заведений по районам

Добавим в таблицу mo_area информацию о количестве заведений по муниципальным районам. И посмотрим в каких муниципальных районах Москвы больше всего точек общественного питания

In [ ]:
mo_area = mo_area.sort_values(by=['ward', 'area'])
list_tmp = rest.groupby(['ward', 'area']).name.count().values
mo_area['name_cnt'] = list_tmp
display(mo_area.sort_values(by='name_cnt', ascending=False).head(5))

По топ-5 районов по количеству заведений видим, что сюда попали как большие по площади районы (Пресненский, Хамовники), так и районы поменьше (Тверской, Таганка). При этом количество точке для них сравнимо. 

Добавим в таблицу информацию о плотности наших заведений, т.е. количество пунктов общественного питания на единицу площади и на 10 000 человек

In [ ]:
mo_area['rest_per_sq'] = round(mo_area.name_cnt / mo_area.square, 2)
mo_area['rest_per_p']= round(10000*mo_area.name_cnt / mo_area.population, 2)

#### Хороплёт по плотности заведений

Визуализируем полученные данные с помощью Хороплёта библиотек Geopandas и Folium. Такой интерактивный график поможет нам найти районы с наименьшей плотностью, но наиболее близкие к центру города. 

In [ ]:
# скопируем два столбца с плотностью в геодатафрейм для отображения на хороплете
df_tmp = mo_df.merge(mo_area[['area', 'rest_per_sq', 'rest_per_p']])

# расставим столбцы в логичном порядке для отображения в легендах к районам
df_tmp = df_tmp[['ward', 'area', 'rest_per_sq', 'rest_per_p', 'geometry']]
# переименуем столбцы
df_tmp.columns = ['Округ:', 'Район:', '- на 1 кв.км', 
                  '- на 10 тыс.жителей', 'geometry']
# добавим пустой столбец просто для изменения легенды
# (не хватило времени разобраться как это сделать по-другому)
df_tmp.insert(2, 'Количество заведений:', ['']*len(df_tmp))

# построим Хороплёт
df_tmp.explore(
    # выбор сделаем по количеству на 1 кв.км
    column="- на 1 кв.км",
    # добавим вывод легенды при нажатии 
    popup = True, cmap="Blues", 
    style_kwds=dict(color=cat_color[5], weight=1, fillOpacity=1),
    highlight_kwds=dict(color=cat_color[1], weight=2),
    zoom_start=10)

- Если пройтись по этой карте, то можно отметить несколько районов, с небольшой плотностью заведений и не сильно удалённых от центра. Например **Раменки**, **Нагорный**, **Тимирязевский** муниципальные округа. 
- Что касается мест, расположенных ближе к МКАДу, то это тоже неплохой вариант, но нужно внимательно выбрать место. Не внутри "спального" района, откуда все разъезжаются по утрам, а вблизи крупных офисов или ВУЗов, около метро, в торговых центрах, т.е. там, где будет больший поток посетителей. Например, **Ясенево**, **Строгино** и т.п. 

#### Количество заведений на 1 квадратный километр

Просто из интереса выведем районы, где проще всего найти где поесть. И те, где заведений меньше всего. 

In [ ]:
list_tmp = ['area', 'ward', 'name_cnt', 'rest_per_sq', 'rest_per_p']
display(mo_area.sort_values(by='rest_per_sq', ascending=False)[list_tmp]
        .reset_index(drop=True).head(5)
        .style.background_gradient('Reds')
        .format('{:0.2f}', subset=['rest_per_sq', 'rest_per_p']))

И аналогично - наименее популярные районы.

In [ ]:
display(mo_area.sort_values(by='rest_per_p', ascending=False)[list_tmp]
        .reset_index(drop=True).tail(15)
        .style.background_gradient('Blues')
        .format('{:0.2f}', subset=['rest_per_sq', 'rest_per_p']))

Обращаем опять внимание на Ясенево и Строгино. 

#### Карта плотности заведений 

К сожалению, хороплёт не получится прикрепить к презентации. Сделаем карту, которую можно будет добавить в файл `pdf`.

In [ ]:
# cоздадим временный геодатафрейм
df_tmp = mo_df.merge(mo_area[['area', 'rest_per_sq']])

# создадим карту
ax = df_tmp.plot(figsize=(20,10), column='rest_per_sq', cmap='Reds', legend=True)
# добавим граинцы
mo_df.boundary.plot(ax=ax, color=cat_color[4]);
# отключим оси
ax.set_axis_off()
# шрифт уменьшим
mpl.rc('font', size=12)

plt.xlim([37.31,37.95])
plt.ylim([55.55,55.97])

plt.title('Количество заведений на 1 кв.км по муниципальным районам')
plt.show();

Думаю, что при выборе места для кофейни лучше будет выбирать районы ближе к центру, но с меньшей плотностью. Уже выбрав возможные варианты можно будет ориентироваться на другие параметры.  

#### Количество заведений на 10 тысяч жителей

Построим аналогичную карту по плотности заведений на количество жителей

In [ ]:
# cоздадим временный геодатафрейм
df_tmp = mo_df.merge(mo_area[['area', 'rest_per_p']])

# создадим карту
ax = df_tmp.plot(
     figsize=(20,10), column='rest_per_p', cmap='Blues', legend=True)
# добавим граинцы
mo_df.boundary.plot(ax=ax, color=cat_color[5]);
# отключим оси
ax.set_axis_off()
# шрифт уменьшим
mpl.rc('font', size=12)

plt.xlim([37.31,37.95])
plt.ylim([55.55,55.97])

plt.title('Количество заведений на 10 тысяч жителей')
plt.show();

###  Количество точек питания по улицам

#### Топ-15 улиц

Аналогично тому, как мы строили тепловую карту по административным округам, построим распределение количества заведений по наиболее популярным улицам.

In [ ]:
# найдём список топ-15 улиц, отсортируем по алфавиту
top_list = sorted(rest.street.value_counts().to_frame().head(15).index)

# рассчитаем нужные данные и запишем в таблицу str_df
df_tmp = pd.DataFrame(index=top_list)

for cat in cat_list:
    list_tmp = []
    for ts in top_list:
        list_tmp += [len(rest.query('category==@cat & street==@ts'))]
    df_tmp[cat] = list_tmp
    
# построим тепловую карту    
plt.figure(figsize=(16, 7))
sns.heatmap(df_tmp, cmap=sns.color_palette("Reds", as_cmap=True), annot=True, fmt='') 

plt.xlabel('')
plt.title('Распределение категорий точек питания по топ-15 улиц') 
plt.show();    

Самая "горячая" точка тепловой карты - количество кафе на проспекте Мира. Это ни о чем не говорит, потому что проспект Мира очень длинный. По расстоянию от одного кафе до другого скорее всего выиграет улица Миклухо-Маклая, пройти которую от начала до конца пешком можно минут за 40. А проиграет, наверное МКАД. 

#### Улицы с одной точкой питания

In [ ]:
value = (rest.street.value_counts()==1).sum()
print(f'На \033[1m{value}\033[0m улицах Москвы расположен только один пункт питания.')

Да, и такие улицы бывают. Короткие переулки в центре города, улицы подлиннее в спальных районах. Как мы видели на хороплёте, есть районы, где очень мало заведений. Улицы с окраин города вполне могли оказаться в этом списке.

А вот в качестве места для расположения нового заведения эти улицы могут и не подойти:
- если это "короткая" улица и там достаточно одного заведения;
- если это какая-то шумная дорога или т.п., то все уютные кафе или рестораны расположены по-близости в переулках;
- если это промышленный или другой "неблагополучный" для размещения заведения район и т.д. 

Если изучить карту с кластерами, то можно найти улицы, где вообще ни одного заведения нет. Как говорила моя бабушка (хотя и немного по другому поводу): "И даже в таких местах люди живут!" 😱

### Величина среднего чека

#### Размер среднего чека по муниципальным районам

Рассчитаем медианные значения среднего чека по округам. Вспомним, что мы заменили пропуски нулями. Поэтому исключим нулевые значения. Затем построим хороплёт, как мы делали для рейтинга.

In [ ]:
bill_df = rest.query('avg_bill>0').groupby('district').avg_bill.median().astype(int)
bill_df = bill_df.sort_values(ascending=False).to_frame().reset_index()
display(bill_df.style.background_gradient('Blues'))

In [ ]:
# создаём карту Москвы
m = folium.Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
folium.Choropleth(geo_data = ao_json, data = bill_df,  columns = ['district', 'avg_bill'],
    key_on = 'feature.name', fill_color = 'Blues', fill_opacity = .8,
    legend_name='Средний рейтинг заведений по округам'
).add_to(m)

# выводим карту
m

- По ценам опередили всех Центральный и Западный округа. Эти административные единици исторически и по стоимости на жильё всегда были самыми дорогими в Москве. Поэтому и хороших "крутых" заведений здесь больше. И цены соответствующие.
- Если построить такой же график не по медиане, а по средним значениям, то ЦАО окажется на твёрдом первом месте. 

### Итоги анализа данных

<div style="color:navy"><b>По количеству точек общественного питания:</b></div>

- проанализированы данные о **8 399** заведениях Москвы;


- около **37%** из них принадлежат какой-либо сети;


- более **33%** - предположительно находятся в торговых центрах и используют общие столики фуд-кортов;


- почти половина всех точек питания - это **рестораны и кафе**, третья по популярности категория - **кофейни**;


- приблизительно так же распределено деление на категории среди сетевых заведений;


- чаще всего сетевыми становятся **булочные, пиццерии и кофейни** - половина из них открыта под маркой одной из сетей;


- больше всего заведений имеют сети **Шоколадница, Домино'с пицца и ДоДо Пицца**;


- меньше всего на рынке общепита представлено **столовых** - их доля от всех заведений не дотягивает до **4%**, столовые почти никогда не бывают круглосуточными и реже всех располагаются в фуд-кортах;


- **круглосуточно** работают в основном **пункты быстрого питания**, они же чаще всего открывают свои заведения в торговых центрах.



<div style="color:navy"><b>По расположению:</b></div>

- больше всего точек расположилось в **ЦАО**, там же находятся и самые дорогие заведения с самым лучшим рейтингом (медиана рейтинга 4.8);


- следующим по популярности административным округом стал **Северный**, но по количеству заведений и он сильно отстаёт от Центрального округа (897 против 2240 в ЦАО);

- вторым по "дороговизне" после Центрального района оказался **ЗАО**;


- меньше всего пунктов питания на **Северо-Западе**, но нельзя сказать, что этот округ наименее популярен, так как он самый маленький по площади из всех округов;

- на юге и юго-востоке города чаще встречаются более дешевые заведения с более низким рейтингом, чем на Севере и Северо-западе;


- если сравнивать районы города по "плотности" точек, то на первое место вышел **Арбат**, где на 1 кв.км можно насчитать почти 58 заведений, а на 10 тысяч жителей этого района приходится около 34 точек питания;


- совсем немного от Арбата отстало **Замоскворече** (55 на 1 кв.км и 43 заведения на 10 тыс.жителей).

<hr style="background-color:navy; align:left; height:2pt">

Учитывая сильную конкуренцию в центре города, большое количество популярных заведений с высоким рейтингом и высокими ценами, открывать новую точку лучше чуть дальше от центра. Анализ распределения заведений по муниципальным округам позволил предположить, что в качестве перспективных районов могли бы стать **Тимирязевский** (чуть больше 4 заведений на 1 кв.км), **Нагорный** (3.8) или **Раменки** (6.5). 

Если выбирать районы с ещё меньшей плотностью расположения пунктов питания, то это будут так называемые "спальные" районы. Среди таких можно отметить **Ясенево** (где всего лишь 2.8 точки на 1 кв.км и чуть больше 4 на 10 тыс.жителей), **Строгино** (3.6 и 3.7 соответственно). Стоит правда учесть, что именно в этих двух районах большую часть занимают природные парки. Возможно что на низкие значения мог повлиять и этот факт. 

<h2 style="color:navy">Открытие кофейни</h2>

Клиент больше склоняется открыть дорогую кофейню. Поэтому отдельно проведём анализ только для таких заведений. Учитывая, что намерение клиента - именно дорогая элитная кофейня, вряд ли стоит рассматривать районы сильно удалённые от центра города. Дело не только в уровне цен. Но и в количестве потенциальных клиентов, которых больше в центре города, в районе скопления бизнес-центров, в туристических местах города. 

### Информация о кофейнях

#### Подсчёт количества 

In [ ]:
# создадим срез по кофейням
coffee_df = rest.query('category=="кофейня"')[['name', 'ward', 'area', 
            'rating', 'chain', 'food_court', 'is_24/7', 
            'prices', 'coffee', 'avg_bill']].reset_index(drop=True)

value = len(coffee_df)
print(f'Количество кофеен \033[1m{value:,}\033[0m. Из них:'.replace(',', ' '))
value = len(coffee_df.query('chain'))/len(coffee_df)
print(f'- сетевых \033[1m{value:0.1%}\033[0m;')
value = coffee_df['is_24/7'].sum()/len(coffee_df)
print(f'- работает круглосуточно \033[1m{value:0.1%}\033[0m;')
value = len(coffee_df.query('food_court'))/len(coffee_df)
print(f'- предположительно расположены в фуд-корте \033[1m{value:0.1%}\033[0m.')

#### Рейтинг кофеен

Посмотрим, насколько сильно отличается средний рейтинг кофеен по административным округам.

In [ ]:
display(coffee_df.groupby('ward').rating.mean().to_frame()
        .sort_values(by='rating', ascending=False).T
        .style.background_gradient('Blues', axis=1).format('{:.2f}'))

В очередной раз убеждаемся, что в среднем в Центральном округе рейтинг кофеен выше, чем в остальных округах. Но вряд ли можно как-то использовать эту информацию. Рейтинг зависит от многих факторов, но прежде всего от качества обслуживания и качества приготовленной еды и напитков. Нет оснований искать здесь какую-то сильную зависимость.

#### Уровень цен в кофейнях

Приблизительно такая же ситуация с уровнем цен. 

In [ ]:
display(coffee_df.query('prices>0').groupby('ward').prices.mean().to_frame()
        .sort_values(by='prices', ascending=False).T
        .style.background_gradient('Blues', axis=1).format('{:.2f}'))

Напомним, что уровень цен равен 0, если цена не указана (эти значения мы не учитывали при анализе), 1 соответствуют низкие цены, а 4 - высокие. 
- Второе место по уровню цен занял Западный округ. Но он неожиданно оказался последним по среднему рейтингу заведений. Может, клиенты тем и недовольны, что цены высоковаты. 
- В центре и на северо-западе города цены в кофейнях выше. На юге и юго-востоке располжились точки питания попроще. 

### Кофейни по административным округам

#### Аггрегированные данные по округам

Создадим таблицу `ao_df` по административным округам для кофеен.

In [ ]:
# используем готовую таблицу ward_df
ao_df = ward_df[['name_cnt', 'color', 'кофейня']].reset_index()
# переименуем столбцы
ao_df.columns = ['ward', 'name_cnt', 'color', 'cof_cnt']

# добавим медиану цены за чашку кофе, отбросив те строки, где цена не указана
ao_df = ao_df.merge(coffee_df.query('coffee>0').groupby('ward').coffee.median().to_frame(),
                    on='ward', how='left')
# добавим медиану среднего чека, отбросив те строки, где цена не указана
ao_df = ao_df.merge(coffee_df.query('avg_bill>0').groupby('ward').avg_bill.median().to_frame(),
                    on='ward', how='left')

# заполним нулями возникшие пропуски 
ao_df = ao_df.fillna(0)
# переведём цены в целое
ao_df[['coffee', 'avg_bill']] = ao_df[['coffee', 'avg_bill']].astype(int)

# рассчитаем процент кофеен от всех заведений по округам
ao_df['cof_pct'] = round(ao_df.cof_cnt / ao_df.name_cnt, 4)

display(ao_df[ao_df.columns.drop('color')]
        .sort_values(by='cof_cnt', ascending=False)
        .style.background_gradient('Reds', subset=['cof_cnt', 'cof_pct', 'coffee', 'avg_bill'])
        .format({'cof_pct': '{:0.2%}'}))

- Для расположения хорошей кофейни Центральный округ подходит плохо ввиду большой конкуретности. Хотя потребность в кофейнях в центре выше, чем на окраинах города. Т.к. в центре больше офисов, люди чаще встречаются в центре, чтобы провести время в кафе или кофейне вместе. 
- Что касается количества заведений по округам, то эта информация несёт в себе мало пользы, т.к. всё зависит ещё и от площади округа. 
- Для определения средней цены за чашку кофе можно будет полагаться на средние значения в зависимости от выбранного района.

#### Доля кофеен по округам

Визуализируем полученные результаты

In [ ]:
# отсортируем по количеству заведений
df_tmp = ao_df[['ward', 'name_cnt', 'cof_cnt', 'cof_pct', 'color']].sort_values(by='cof_pct')
df_tmp = df_tmp.set_index('ward')

# построим диаграмму по количеству заведений
fig = df_tmp.name_cnt.plot.barh(figsize=(10,5), xlim=(0,1000), width=.8, 
        xlabel='', color=df_tmp['color'], alpha=0.5)

# на тех же осях добавим количество кофеен
fig = df_tmp.cof_cnt.plot.barh(width=.8, xlabel='', color=df_tmp['color'], ax=fig)

# подпишем количества
i = -.2
for row in df_tmp.index:
    n1 = df_tmp.loc[row, 'cof_cnt']
    n2 = round(df_tmp.loc[row, 'cof_pct'] * 100)
    n3 = df_tmp.loc[row, 'name_cnt']
    value = str(n2) + '% (' + str(n1) + ')'
    # подпись процентов и количества кофеен
    plt.text(n1+20, i, value, horizontalalignment='left')
    # подпись общего количества заведений
    if row=='ЦАО': n1 = 1000
    else: n1 = n3
    plt.text(n1-20, i, n3, horizontalalignment='right')
    i+=1

# скроем верхнюю и правую линии на осях
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)    
    
plt.title('Количество и доли кофеен среди заведений всех категорий', size=14)
plt.xlabel('количество заведений')
plt.show();

Получается, что в ЮЗАО, ВАО и ЮВАО кофеен может быть не достаточно.

#### Величина среднего чека и цена чашки капучино

In [ ]:
# отсортируем по цене за чашку
df_tmp = ao_df.sort_values(by='coffee', ascending=False)[['ward', 'coffee', 'avg_bill', 'color']]
df_tmp = df_tmp.set_index('ward')

# построим диаграмму по среднему чеку
fig = df_tmp.avg_bill.plot.bar(figsize=(10,7), width=.8, 
        xlabel='', color=df_tmp['color'], alpha=0.5)

# добавим цену за чашку кофе на тех же осях
df_tmp.coffee.plot.bar(width=.8, xlabel='', color=df_tmp['color'], ax=fig)

# подпишем цены
i = 0.2
for row in df_tmp.coffee:
    plt.text(i, row-20, row, horizontalalignment='right', color='white')
    i+=1
i = 0.2
for row in df_tmp.avg_bill:
    plt.text(i, row-20, row, horizontalalignment='right')
    i+=1
    
# скроем верхнюю и правую линии на осях
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
    
plt.title('Величина среднего чека\nи медиана средней цены за чашку капучино в кофейнях', size=14)
plt.ylabel('цена, руб.')
plt.xticks(rotation = 0) 
plt.show();

В центре, на западе и юго-западе Москвы выпить кофе в среднем обойдётся дороже, чем на юге, востоке и юго-востоке. 

#### Аггрегированные данные по районам

Оставим в срезе `coffee_df` только общие данные по районам.

In [ ]:
# подсчитаем количество кофеен и средний рейтинг
df_tmp = coffee_df.groupby('area').agg({'name': 'count', 'rating': 'mean'}).reset_index()
df_tmp.columns = ['area', 'cof_cnt', 'rating']

# используем готовую таблицу mo_area
df_tmp = df_tmp.merge(mo_area[['ward', 'area', 'square', 'population', 'name_cnt']], 
                                     on='area', how='left')

# добавим медиану цены за чашку кофе, отбросив те строки, где цена не указана
df_tmp = df_tmp.merge(coffee_df.query('coffee>0').groupby('area').coffee.median()
                      .to_frame(), on='area', how='left')
# добавим медиану среднего чека, отбросив те строки, где цена не указана
df_tmp = df_tmp.merge(coffee_df.query('avg_bill>0').groupby('area').avg_bill.median()
                      .to_frame(), on='area', how='left')

# рассчитаем доли кофеен и "плотность"
df_tmp['cof_pct'] = round(100 * df_tmp.cof_cnt / df_tmp.name_cnt, 2)
df_tmp['cof_per_sq'] = round(df_tmp.cof_cnt / df_tmp.square, 2)
df_tmp['cof_per_p'] = round(10000 * df_tmp.cof_cnt / df_tmp.population, 2)

# заполним нулями возникшие пропуски 
df_tmp=df_tmp.fillna(0)

# округлим рейтинг до двух знаков
df_tmp.rating = round(df_tmp.rating , 2)
# переведём уровень цен и средний чек в целое
df_tmp[['coffee', 'avg_bill']] = df_tmp[['coffee', 'avg_bill']].astype(int)

# пересохраним аггрегированные данные в таблицу coffee_df, оставив только нужные столбцы в нужном порядке
coffee_df = df_tmp[['area', 'ward', 'cof_per_sq', 'cof_per_p', 'cof_pct', 'rating', 'coffee', 'avg_bill']]

#### Топ-5 районов 

Выведем информацию о топ-5 районах по разным параметрам.

In [ ]:
%%capture --no-display
# добавим столбец для подписей
coffee_df['name'] = coffee_df.area + ' (' + coffee_df.ward + ')'

In [ ]:
list_tmp = coffee_df.sort_values(by='cof_per_sq', ascending=False).head(5).name.to_list()
print('По количеству кофеен на единицу площади:\033[1m')
print(*list_tmp, sep=', ')
print('\033[0m')

list_tmp = coffee_df.sort_values(by='cof_per_p', ascending=False).head(5).name.to_list()
print('По количеству кофеен на 10 тыс. жителей:\033[1m')
print(*list_tmp, sep=', ')
print('\033[0m')

list_tmp = coffee_df.sort_values(by='cof_pct', ascending=False).head(5).name.to_list()
print('По доле кофеен среди других заведений:\033[1m')
print(*list_tmp, sep=', ')
print('\033[0m')

list_tmp = coffee_df.sort_values(by='coffee', ascending=False).head(5).name.to_list()
print('По цене за чашку капучино:\033[1m')
print(*list_tmp, sep=', ')
print('\033[0m')

list_tmp = coffee_df.sort_values(by='avg_bill', ascending=False).head(5).name.to_list()
print('По величине среднего чека:\033[1m')
print(*list_tmp, sep=', ')
print('\033[0m')

#### Районы, где кофеен мало

In [ ]:
list_tmp = coffee_df.sort_values(by='cof_per_sq').head(5).name.to_list()
print('По количеству кофеен на единицу площади:\033[1m')
print(*list_tmp, sep=', ')
print('\033[0m')

list_tmp = coffee_df.sort_values(by='cof_per_p').head(5).name.to_list()
print('По количеству кофеен на 10 тыс. жителей:\033[1m')
print(*list_tmp, sep=', ')
print('\033[0m')

list_tmp = coffee_df.sort_values(by='cof_pct').head(5).name.to_list()
print('По доле кофеен среди других заведений:\033[1m')
print(*list_tmp, sep=', ')
print('\033[0m')

In [ ]:
# уберём из coffee_df "лишиние" столбцы
coffee_df = coffee_df[coffee_df.columns.drop(['name', 'ward'])]

### Выбор местоположения для кофейни

#### Визуализация всех параметров по районам

Сделаем интерактивную карту для изучения распределения кофеен по городу.

In [ ]:
# создадим геодатафрейм для кофеен
df_tmp = mo_df.merge(coffee_df, how='left')
# заполним пропуски нулями
df_tmp = df_tmp.fillna(0)
# цены переведём в целоеV
df_tmp[['coffee', 'avg_bill']] = df_tmp[['coffee', 'avg_bill']].astype(int)

# создадим список столбцов для выбора на графике
list_tmp = ['На 1 кв.км', 'На 10 тыс.чел.', 'Доля кофеен', 'Рейтинг', 'Цена чашки', 'Cредний чек']
# переименуем столбцы для отображения
df_tmp.columns = ['Район', 'Округ', 'geometry'] + list_tmp 

In [ ]:
print('\033[1m   Распределение кофеен по районам\033[0m')

# создадим выпадающий список для выбора столбца
@widgets.interact
def interactive_chor(Градиент=list_tmp, Цвет=['Blues', 'Reds', 'PuBu', 'RdBu_r']):
    # построим и выведем на экран Хороплёт
    return df_tmp.explore(
        # выбор сделаем по столбцам из выпадающего списка
        column=Градиент, cmap=Цвет, 
        # добавим вывод легенды при нажатии 
        popup = True, 
        # цвет границ
        style_kwds=dict(color='black', weight=1, fillOpacity=1),
        # цвет подсветки выбранного района
        highlight_kwds=dict(color='yellow', weight=4), 
        # зум при обновлении карты
        zoom_start=10, width=700)

#### Выбор местоположения

При изучении карты по количеству заведений на единицу площади, было отмечено несколько районов, близких к центру, но с относительно невысоким значением "плотности". Были выделены, например, район Раменки, Тимирязевский и др. Из таких районов по средней цене чашки капучино, по расположению, по привлекательности в плане возможного притока клиентов, самым интересным показались **Раменки**. 
Изучив этот район с помощью кластеризации, было найдено два приблизительных варианта улиц для размещения кофейни. Улица Косыгина из Раменок плавно перешла в **Гагаринский** район.

Если открыть кофейню в центре или близко к центру "с нуля" будет сложно по экономическим соображениям, то можно рассмотреть районы ближе к МКАДу, где кофеен либо нет, либо очень мало. Изучение кластеров и хороплёта показало, что **Ясенево** не очень перспективно, т.к. около метро уже есть несколько кофеен, а в районе очень мало точек, где можно будет ждать большого наплыва клиентов - нет ни ВУЗов, ни бизнес-центров, ни достопримечательностей. В этом плане больше подойдёт **Строгино**, где есть факультеты ВШЭ, Дворец спорта "Янтарь",  несколько небольших бизнес-центров и отсутствие кофеен в непосредственной близости к метро.

Подготовим еще одну карту для презентации, выделив выбранные районы. 

In [ ]:
# создадим карту
ax = mo_df.plot(
     figsize=(20,10), color=cat_color[4], alpha=.1,
     legend=True, legend_kwds={"location":"left","shrink":.5})

# добавим выделение выбранных районов
mo_df.query('area.str.contains("Гагаринский")').plot(
            color=cat_color[5], ax=ax, label='Гагаринский')
mo_df.query('area.str.contains("Раменки")').plot(
            color=cat_color[5], alpha=.5, ax=ax, label='Раменки')
mo_df.query('area.str.contains("Строгино")').plot(
            color=cat_color[2], alpha=.5, ax=ax, label='Раменки')


# нарисуем границы
mo_df.boundary.plot(ax=ax, color=cat_color[4]);

# отключим оси
ax.set_axis_off()
# шрифт уменьшим
mpl.rc('font', size=12)

plt.xlim([37.31,37.95])
plt.ylim([55.55,55.97])

plt.title('Выбор района для кофейни')
plt.show();

### Итоги изучения кофеен

<div style="color:navy"><b>По количеству:</b></div>

- проанализированы данные о **1 413** кофейнях Москвы; 


- более половины из них принадлежат какой-либо сети, самыми популярными сетевыми кофейнями являются
    - **Шоколадница**, 
    - **OnePrice Coffee**,
    - **Coffix**;


- только **4.2%** кофеен работают круглосуточно.

<div style="color:navy"><b>По расположению:</b></div>

- больше всего заведений с хорошим рейтингом, высокими ценами в **Центральном** административном округе;


- любят пить кофе в **САО** - там самая высокая доля кофеен среди остальных заведений;


- на **юге, юго-востоке и востоке** города ситуация чуть хуже - кофеен там мало, средний рейтинг и медианы цен ниже;


- в Москве существуют районы, где нет ни одной кофейни. К сожалению, это муниципальные округа на окраине города, т.е "спальные" районы. Например, **Бирюлёво-Западное**. Целесообразность открытия там кофейни сомнительна из-за возможного недостатка в клиентах.

<hr style="background-color:navy; align:left; height:2pt">

С помощью графической визуализации (хороплётов и кластеризации) выявлено несколько районов, где можно было бы расположить новую кофейню. Стоит иметь ввиду, что анализа картографических данных для окончательного принятия решения не достаточно. Следует обязательно учесть маркетинговые особенности, вероятный траффик клиентов, уровни цен, логистики и т.д. и т.п.


1. **Улица Косыгина**, Гагаринский муниципальный округ (ЮЗАО) и Раменки (ЗАО)
    - близко от центра, лёгкая транспортная доступность (м.Ленинский проспект, м.Университет);
    - наличие достопримечательностей и парков рядом, близость ВУЗов, Дом детского творчества, Академия Наук,  комплекс зимних видов спорта и т.п.;
    - количество кофеен в районе выше среднего, но именно кофейни на указанной улице и по близости отсутствуют;
    - высокая средняя цена за чашку кофе (227 руб. в Гагаринском районе) и средний чек (500 руб.);
    - скорое открытие штаб-квартиры Яндекса на улице Косыгина 🙂!


2. **Раменки (ЗАО)**, улицы: Светланова, Шувалова, Раменский бульвар
    - не очень далеко от центра города;
    - относительно низкое количество кофеен на единицу площади и на количество жителей района;
    - высокая средняя цена за чашку кофе (256 руб.) и средний чек по кофейням района (400 руб.);
    - близкое расположение корпусов и общежитий МГУ, а также элитной недвижимости в этом районе (возможная постоянная клиентура);
    - строящиеся объекты на новой территории МГУ (также возможность дополнительного привлечения посетителей);
    - наличие парков и скверов в пешеходной доступности (многие заходят в перекусить после прогулки или по дороге).
    

3. Район около метро **Строгино** можно рассмотреть как вариант стартапа с планом развития в более дорогое заведение ближе к центру города
    - очень маленькое количество кофеен как на единицу площади, так и на количество жителей (ниже чем 0.5 на 1 кв.км и 10 тыс. жителей);
    - огромная природная и парковая территория в районе;
    - наличие в районе ВУЗа, спортивного комплекса, бизнес-центра (возможны клиенты "по дороге на работу");
    - неплохая для районов в отдалении от центра средняя цена чашки кофе (220 руб.).

<hr style="background-color:navy; align:left; height:2pt">

**Презентация**: <https://drive.google.com/file/d/1w6m9h8FZNUbWVFA-4QIve5qZkGm2oBx-/view> 